In [1]:
import numpy as np
from collections import OrderedDict
from scipy.stats import norm, uniform

In [2]:
def __randnetbalanced(dims, samples, indegree, parminmax, errminmax, random_state):
    """
    create a more balanced random network

    Parameter
    ---------
    dims : int
        number of variables
    samples : int
        number of samples
    indegree : int or float('inf')
        number of parents of each node (float('inf') = fully connected)
    parminmax : dictionary
        standard deviation owing to parents 
    errminmax : dictionary
        standard deviation owing to error variable
    random_state : np.random.RandomState
        random state

    Return
    ------
    B : array, shape (dims, dims)
        the strictly lower triangular network matrix
    errstd : array, shape (dims, 1)
        the vector of error (disturbance) standard deviations
    """

    # First, generate errstd
    errstd = uniform.rvs(loc=errminmax['min'], scale=errminmax['max'], size=[
                         dims, 1], random_state=random_state)

    # Initializations
    X = np.empty(shape=[dims, samples])
    B = np.zeros([dims, dims])

    # Go trough each node in turn
    for i in range(dims):

        # If indegree is finite, randomly pick that many parents,
        # else, all previous variables are parents
        if indegree == float('inf'):
            if i <= indegree:
                par = np.arange(i)
            else:
                par = random_state.permutation(i)[:indegree]
        else:
            par = np.arange(i)

        if len(par) == 0:
            # if node has no parents
            # Increase errstd to get it to roughly same variance
            parent_std = uniform.rvs(
                loc=parminmax['min'], scale=parminmax['max'], random_state=random_state)
            errstd[i] = np.sqrt(errstd[i]**2 + parent_std**2)

            # Set data matrix to empty
            X[i] = np.zeros(samples)
        else:
            # If node has parents, do the following
            w = norm.rvs(size=[1, len(par)], random_state=random_state)

            # Randomly pick weights
            wfull = np.zeros([1, i])
            wfull[0, par] = w

            # Calculate contribution of parents
            X[i] = np.dot(wfull, X[:i, :])

            # Randomly select a 'parents std'
            parstd = uniform.rvs(
                loc=parminmax['min'], scale=parminmax['max'], random_state=random_state)

            # Scale w so that the combination of parents has 'parstd' std
            scaling = parstd / np.sqrt(np.mean(X[i] ** 2))
            w = w * scaling

            # Recalculate contribution of parents
            wfull = np.zeros([1, i])
            wfull[0, par] = w
            X[i] = np.dot(wfull, X[:i, :])

            # Fill in B
            B[i, par] = w

        # Update data matrix
        X[i] = X[i] + norm.rvs(size=samples,
                               random_state=random_state) * errstd[i]

    return B, errstd

In [3]:
def generate_data(dims=6, samples=10000, seed=None):
    """
    Parameter
    ---------
    dims : int
        number of variables
    samples : int
        number of samples
    seed : int
        seed for random_state. Default is None.

    References
    ----------
    .. [1] SS. Shimizu, P. O. Hoyer, A. Hyvärinen, and A. J. Kerminen.
       A linear non-gaussian acyclic model for causal discovery.
       Journal of Machine Learning Research, 7:2003-2030, 2006.
    .. [2] http://www.cs.helsinki.fi/group/neuroinf/lingam/lingam.tar.gz
       ./lingam-1.4.2/code/randnetbalanced.m
       ./lingam-1.4.2/code/jmlr/plotsjmlr.m
    """

    random_state = np.random.RandomState(seed=seed)

    # Randomly select sparse/full connections
    sparse = np.floor(uniform.rvs(random_state=random_state) * 2).astype(int)
    indegree = (np.floor(uniform.rvs(random_state=random_state) *
                         3).astype(int) + 1) if sparse == 1 else float('inf')

    # Create the network
    B, disturbancestd = __randnetbalanced(dims, samples, indegree, {
                                          'min': 0.5, 'max': 1.5}, {'min': 0.5, 'max': 1.5}, random_state)

    # constants, giving non-zero means
    c = 2 * norm.rvs(size=[dims, 1], random_state=random_state)

    # nonlinearity exponent, in [0.5, 0.8] or [1.2, 2.0].
    q = uniform.rvs(size=[dims, 1], random_state=random_state) * 1.1 + 0.5
    index = q > 0.8
    q[index] = q[index] + 0.4

    # This generates the disturbance variables, which are mutually
    # independent, and non-gaussian
    S = norm.rvs(size=[dims, samples], random_state=random_state)
    S = np.sign(S) * (np.abs(S) ** np.dot(q, np.ones([1, samples])))

    # This normalizes the disturbance variables to have the
    # appropriate scales
    S = S / ((np.sqrt(np.mean(S.T**2).T) / disturbancestd)
             * np.ones([1, samples]))

    # Now we generate the data one component at a time
    Xorig = np.zeros([dims, samples])
    for i in range(dims):
        Xorig[i] = np.dot(B[i], Xorig) + S[i] + c[i]

    # Select a random permutation because we do not assume that
    # we know the correct ordering of the variables
    p = random_state.permutation(dims)

    # Permute the rows of the data matrix, to give us the
    # observed data
    X = Xorig[p]

    # Permute the rows and columns of the original generating
    # matrix B so that they correspond to the actual data
    Bp = B[p][:, p]

    # Permute the generating disturbance stds so that they
    # correspond to the actual data
    disturbancestdp = disturbancestd[p]

    # Permute the generating constants so that they correspond to
    # the actual data
    cp = c[p]

    # make causal order
    causal_order = np.empty(len(p))
    causal_order[p] = np.arange(len(p))

    return X.T, Bp, causal_order

In [4]:
def make_test_data(dims_list, samples_list, data_set_num, set_seed=True):
    """ make some data sets to test.
    """
    test_data_set = OrderedDict()

    count = 0
    for dims in dims_list:
        for samples in samples_list:
            test_data_set[(dims, samples)] = []

            for data_no in range(data_set_num):
                X, Bp, causal_order = generate_data(
                    dims=dims, samples=samples, seed=(count if set_seed else None))
                test_data_set[(dims, samples)].append({'X': X, 'B': Bp, 'causal_order': causal_order})

                count += 1

    return test_data_set

## Generate Datasets

In [5]:
dims_list = [10, 50, 100]
samples_list = [200, 1000, 5000]
data_set_num = 10

test_data_set = make_test_data(dims_list, samples_list, data_set_num)